# TP 7 : Manipulation de tenseurs avec Pytorch

In [13]:
# NOM : Andrieu
# Prénom : Ludovic
# N° étudiant : 22103219

In [2]:
import torch
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

L'objectif de cette séance est de se familiariser avec les manipulations tensorielles de base dans PyTorch, de comprendre la relation entre un tenseur et son stockage sous-jacent, et d'apprécier l'efficacité du calcul tensoriel par rapport à ses équivalents itératifs en Python.

## Exercice 1

Générer la matrice suivante sans utiliser de boucle Python :

```
1 2 1 1 1 1 2 1 1 1 1 2 1
2 2 2 2 2 2 2 2 2 2 2 2 2
1 2 1 1 1 1 2 1 1 1 1 2 1
1 2 1 3 3 1 2 1 3 3 1 2 1
1 2 1 3 3 1 2 1 3 3 1 2 1
1 2 1 1 1 1 2 1 1 1 1 2 1
2 2 2 2 2 2 2 2 2 2 2 2 2
1 2 1 1 1 1 2 1 1 1 1 2 1
1 2 1 3 3 1 2 1 3 3 1 2 1
1 2 1 3 3 1 2 1 3 3 1 2 1
1 2 1 1 1 1 2 1 1 1 1 2 1
2 2 2 2 2 2 2 2 2 2 2 2 2
1 2 1 1 1 1 2 1 1 1 1 2 1
```

**Indice :** Utiliser `torch.full` et le `slicing`

In [3]:
t = torch.full((13, 13), 1)
t[1: : 5] = 2
t[:, 1: : 5] = 2
t[3:: 5, 3:: 5] = 3
t[4:: 5, 3:: 5] = 3
t[3:: 5, 4:: 5] = 3
t[4:: 5, 4:: 5] = 3
t

tensor([[1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [1, 2, 1, 3, 3, 1, 2, 1, 3, 3, 1, 2, 1],
        [1, 2, 1, 3, 3, 1, 2, 1, 3, 3, 1, 2, 1],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [1, 2, 1, 3, 3, 1, 2, 1, 3, 3, 1, 2, 1],
        [1, 2, 1, 3, 3, 1, 2, 1, 3, 3, 1, 2, 1],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1]])

## Exercice 2

Générer deux matrices carrées de dimension $5000 \times 5000$ remplies de coefficients gaussiens aléatoires, calculer leur produit, mesurer le temps nécessaire et estimer le nombre de produits en virgule flottante exécutés par seconde (devrait être de l'ordre de milliards ou dizaines de milliards).

**Indice :** Utiliser `torch.empty`, `torch.normal`, `torch.mm` et `time.perf_counter`.

In [4]:
N = 5000
a = torch.normal(torch.empty(N, N)).to(device)
b = torch.normal(torch.empty(N, N)).to(device)
tStart = time.perf_counter()
_ = a @ b
duration = time.perf_counter() - tStart
del _, a, b
nbOps = N ** 3
print(f"on fait {nbOps/duration/1e12:_.3f} TOPS/sec")

on fait 2.678 TOPS/sec


## Exercice 3

Écrire une fonction `mul_row`, en utilisant des boucles Python (et pas de slicing), qui prend un tenseur 2D en argument et renvoie un tenseur de même taille, dont la première ligne est identique à la première ligne du tenseur d'argument, la deuxième ligne est multipliée par deux, la troisième par trois, etc.

**Exemple :**

```python
>>> torch.full((4, 8), 2.0)
>>> m
tensor([[2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2.]])
>>> mul_row(m)
tensor([[2., 2., 2., 2., 2., 2., 2., 2.],
        [4., 4., 4., 4., 4., 4., 4., 4.],
        [6., 6., 6., 6., 6., 6., 6., 6.],
        [8., 8., 8., 8., 8., 8., 8., 8.]])
```

Ensuite, écrire une seconde version nommée `mul_row_fast`, en utilisant des opérations tensorielles.

Appliquer les deux versions à une matrice de taille $1000 \times 400$ et mesurer le temps nécessaire pour chacune (il devrait y avoir plus de deux ordres de grandeur de différence).

**Indice :** Utiliser le `broadcasting` et `torch.arange`, `torch.view`, `torch.mul` et `time.perf_counter`.

In [5]:
def mul_row(t:torch.Tensor)->torch.Tensor:
    result = torch.empty_like(t)
    nbRows, nbCols = t.shape
    for row in range(nbRows):
        for col in range(nbCols):
            result[row, col] = t[row, col] * (1+row)
    return result

In [6]:
def mul_row_fast(t:torch.Tensor)->torch.Tensor:
    nbRows, nbCols = t.shape
    coeffs = torch.arange(1, nbRows+1, 1).to(t.device)
    result = t * coeffs.view(nbRows, 1)
    return result

In [7]:
print(mul_row(torch.full((4, 8), 2.0)))
print(mul_row_fast(torch.full((4, 8), 2.0)))

tensor([[2., 2., 2., 2., 2., 2., 2., 2.],
        [4., 4., 4., 4., 4., 4., 4., 4.],
        [6., 6., 6., 6., 6., 6., 6., 6.],
        [8., 8., 8., 8., 8., 8., 8., 8.]])
tensor([[2., 2., 2., 2., 2., 2., 2., 2.],
        [4., 4., 4., 4., 4., 4., 4., 4.],
        [6., 6., 6., 6., 6., 6., 6., 6.],
        [8., 8., 8., 8., 8., 8., 8., 8.]])


In [8]:
A = torch.normal(torch.empty((1000, 400)), 1.0)
tStart = time.perf_counter()
_ = mul_row(A)
duration = time.perf_counter() - tStart
print(f"on fait mul_row en {duration:.3f} s")
tStart = time.perf_counter()
_ = mul_row_fast(A)
duration = time.perf_counter() - tStart
print(f"on fait mul_row_fast en {duration*1e6:.3f} µs")
del _


on fait mul_row en 5.988 s
on fait mul_row_fast en 311.400 µs


## Exercice 4

Écrire une fonction qui prend un ensemble d'apprentissage et un échantillon de test et renvoie l'étiquette du point d'apprentissage le plus proche de ce dernier.

Plus précisément, écrire :

```python
def nearest_classification(train_input, train_target, x):
    # Votre code ici
```

où :

* `train_input` est un tenseur 2D de flottants de dimension $n \times d$ contenant les vecteurs d'apprentissage,
* `train_target` est un tenseur 1D de type `long` de dimension $n$ contenant les étiquettes d'apprentissage,
* `x` est un tenseur 2D de flottants de dimension $m \times d$ contenant plusieurs vecteurs de test,

et la valeur renvoyée est la classe de l'échantillon d'apprentissage pour chaque vecteur de `x`, pour la norme $L^2$.

**Indice :** La fonction ne doit pas comporter de boucle Python et peut utiliser notamment `torch.cdist`, `torch.argmin`, `torch.pow`.

In [9]:
def nearest_classification(
        train_input:torch.Tensor, train_target:torch.Tensor,
        x:torch.Tensor)->torch.Tensor:
    dists = torch.cdist(train_input, x, p=2)
    argMin = torch.argmin(dists, dim=0)
    return train_target[argMin]
n = 1000
m = 9
d = 105
train_in = torch.empty((n, d)).normal_()
train_cls = torch.arange(0, n, 1)
test_in = torch.empty((m, d)).normal_()
nearest_classification(train_in, train_cls, test_in)

tensor([ 36, 980,  94, 167, 212, 363, 164,  55, 231])

## Exercice 5

Écrire une fonction :

```python
def compute_nb_errors(train_input, train_target, test_input, test_target, mean=None, proj=None):
    # Votre code ici
```

où :

* `train_input` est un tenseur 2D de flottants de dimension $n \times d$ contenant les vecteurs d'apprentissage,
* `train_target` est un tenseur 1D de type `long` de dimension $n$ contenant les étiquettes d'apprentissage,
* `test_input` est un tenseur 2D de flottants de dimension $m \times d$ contenant les vecteurs de test,
* `test_target` est un tenseur 1D de type `long` de dimension $m$ contenant les étiquettes de test,
* `mean` est soit `None`, soit un tenseur 1D de flottants de dimension $d$,
* `proj` est soit `None`, soit un tenseur 2D de flottants de dimension $c \times d$.

Cette fonction soustrait `mean` (si différent de `None`) aux vecteurs de `train_input` et `test_input`, applique l'opérateur `proj` (si différent de `None`) aux deux, et renvoie le nombre d'erreurs de classification en utilisant la règle du 1-plus proche voisin sur les données résultantes.

In [10]:
def compute_nb_errors(
        train_input:torch.Tensor, train_target:torch.Tensor, 
        test_input:torch.Tensor, test_target:torch.Tensor,
        mean:None|torch.Tensor=None, proj:None|torch.Tensor=None)->int:
    if mean is not None:
        train_input = train_input - mean # not inplace
        test_input = test_input - mean # not inplace
    if proj is not None:
        train_input = train_input @ proj.T  # not inplace
        test_input = test_input @ proj.T # not inplace
    near = nearest_classification(train_input, train_target, test_input)
    errors = (near != test_target)
    return int(errors.sum().item())
    


n = 10000
nbCls = 3
m = 500
d = 15
c = 21

train_in = torch.empty((n, d)).normal_()
train_cls = torch.randint(0, nbCls, (n ,))
test_in = torch.empty((m, d)).normal_()
test_cls = torch.randint(0, nbCls, (m ,))
mean = torch.empty((d, )).normal_(mean=1.5, std=2)
proj = torch.empty((c, d)).normal_(mean=1.0, std=0.54)
compute_nb_errors(train_in, train_cls, test_in, test_cls, mean=mean, proj=proj)

331

## Exercice 6

Comparer les performances de la règle du 1-plus proche voisin sur les données brutes, puis sur les données projetées sur un sous-espace aléatoire de dimension 100 (c'est-à-dire en utilisant une base générée avec une loi normale)

In [11]:
from sklearn.datasets import load_breast_cancer

X, y = load_breast_cancer(return_X_y=True)
X.shape

(569, 30)

In [12]:
from sklearn.model_selection import train_test_split, cross_val_score
import numpy

X_torch = torch.from_numpy(X)
y_torch = torch.from_numpy(y)

scores_with_proj = []
scores_with_proj2 = []
scores_without_proj = []
D = 5

for _ in range(300):
    train_input, test_input, train_target, test_target = \
        train_test_split(X_torch, y_torch, test_size=0.2)
    mean = train_input.mean(dim=0)

    proj = torch.empty((100, X.shape[1]), dtype=torch.float64).normal_(mean=0, std=1.0)
    proj_small = torch.empty((D, X.shape[1]), dtype=torch.float64).normal_(mean=0, std=1.0)

    scores_with_proj.append(
        compute_nb_errors(
            train_input, train_target, test_input,
            test_target, mean=mean, proj=proj))
    scores_with_proj2.append(
        compute_nb_errors(
            train_input, train_target, test_input,
            test_target, mean=mean, proj=proj_small))
    scores_without_proj.append(
        compute_nb_errors(
            train_input, train_target, test_input,
            test_target, mean=mean, proj=None))

print(f"nb of errores with proj: {numpy.mean(scores_with_proj):.3g} +/- {numpy.std(scores_with_proj):.3g}")
print(f"nb of errores with proj(dim={D}): {numpy.mean(scores_with_proj2):.3g} +/- {numpy.std(scores_with_proj2):.3g}")
print(f"nb of errores without proj: {numpy.mean(scores_without_proj):.3g} +/- {numpy.std(scores_without_proj):.3g}")

nb of errores with proj: 10 +/- 2.6
nb of errores with proj(dim=5): 10.5 +/- 2.8
nb of errores without proj: 9.98 +/- 2.65
